In [1]:
"""
This script can be used to evaluate a trained model on 3D pose/shape and masks/part segmentation. You first need to download the datasets and preprocess them.
Example usage:
```
python3 eval.py --checkpoint=data/model_checkpoint.pt --dataset=h36m-p1 --log_freq=20
```
Running the above command will compute the MPJPE and Reconstruction Error on the Human3.6M dataset (Protocol I). The ```--dataset``` option can take different values based on the type of evaluation you want to perform:
1. Human3.6M Protocol 1 ```--dataset=h36m-p1```
2. Human3.6M Protocol 2 ```--dataset=h36m-p2```
3. 3DPW ```--dataset=3dpw```
4. LSP ```--dataset=lsp```
5. MPI-INF-3DHP ```--dataset=mpi-inf-3dhp```
"""

import torch
from torch.utils.data import DataLoader
import numpy as np
import cv2
import os
import argparse
import json
from collections import namedtuple
from tqdm import tqdm
import torchgeometry as tgm

import config
import constants
from models import hmr, SMPL
from datasets import BaseDataset
from utils.imutils import uncrop
from utils.pose_utils import reconstruction_error
from utils.part_utils import PartRenderer

from scipy.spatial.transform import Rotation as R

# Define command-line arguments
parser = argparse.ArgumentParser()
parser.add_argument('--checkpoint', default=None, help='Path to network checkpoint')
parser.add_argument('--dataset', default='h36m-p1', choices=['h36m-p1', 'h36m-p2', 'lsp', '3dpw', 'mpi-inf-3dhp'], help='Choose evaluation dataset')
parser.add_argument('--log_freq', default=50, type=int, help='Frequency of printing intermediate results')
parser.add_argument('--batch_size', default=32, help='Batch size for testing')
parser.add_argument('--shuffle', default=False, action='store_true', help='Shuffle data')
parser.add_argument('--num_workers', default=8, type=int, help='Number of processes for data loading')
parser.add_argument('--result_file', default=None, help='If set, save detections to a .npz file')

def run_evaluation(model, dataset_name, dataset, result_file,
                   batch_size=32, img_res=224, 
                   num_workers=32, shuffle=False, log_freq=50):
    """Run evaluation on the datasets and metrics we report in the paper. """

    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # Transfer model to the GPU
    model.to(device)

    # Load SMPL model
    smpl_neutral = SMPL(config.SMPL_MODEL_DIR,
                        create_transl=False).to(device)
    smpl_male = SMPL(config.SMPL_MODEL_DIR,
                     gender='male',
                     create_transl=False).to(device)
    smpl_female = SMPL(config.SMPL_MODEL_DIR,
                       gender='female',
                       create_transl=False).to(device)
    
    renderer = PartRenderer()
    
    # Regressor for H36m joints
    J_regressor = torch.from_numpy(np.load(config.JOINT_REGRESSOR_H36M)).float()
    
    save_results = result_file is not None
    # Disable shuffling if you want to save the results
    if save_results:
        shuffle=False
    # Create dataloader for the dataset
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
    
    # Pose metrics
    # MPJPE and Reconstruction error for the non-parametric and parametric shapes
    mpjpe = np.zeros(len(dataset))
    recon_err = np.zeros(len(dataset))
    mpjpe_smpl = np.zeros(len(dataset))
    recon_err_smpl = np.zeros(len(dataset))
    
    # Including mean orientation error
    moe = np.zeros(len(dataset))

    # Shape metrics
    # Mean per-vertex error
    shape_err = np.zeros(len(dataset))
    shape_err_smpl = np.zeros(len(dataset))

    # Mask and part metrics
    # Accuracy
    accuracy = 0.
    parts_accuracy = 0.
    # True positive, false positive and false negative
    tp = np.zeros((2,1))
    fp = np.zeros((2,1))
    fn = np.zeros((2,1))
    parts_tp = np.zeros((7,1))
    parts_fp = np.zeros((7,1))
    parts_fn = np.zeros((7,1))
    # Pixel count accumulators
    pixel_count = 0
    parts_pixel_count = 0

    # Store SMPL parameters
    smpl_pose = np.zeros((len(dataset), 72))
    smpl_betas = np.zeros((len(dataset), 10))
    smpl_camera = np.zeros((len(dataset), 3))
    pred_joints = np.zeros((len(dataset), 17, 3))

    eval_pose = False
    eval_masks = False
    eval_parts = False
    eval_orientation = False # Adding the orientation parameter
    # Choose appropriate evaluation for each dataset
    if dataset_name == 'h36m-p1' or dataset_name == 'h36m-p2' or dataset_name == 'mpi-inf-3dhp':
        eval_pose = True
    elif dataset_name == 'lsp':
        eval_masks = True
        eval_parts = True
        annot_path = config.DATASET_FOLDERS['upi-s1h']
    elif dataset_name == '3dpw':
        eval_orientation = True
        eval_pose = True
        

    joint_mapper_h36m = constants.H36M_TO_J17 if dataset_name == 'mpi-inf-3dhp' else constants.H36M_TO_J14
    joint_mapper_gt = constants.J24_TO_J17 if dataset_name == 'mpi-inf-3dhp' else constants.J24_TO_J14
    # Iterate over the entire dataset
    for step, batch in enumerate(tqdm(data_loader, desc='Eval', total=len(data_loader))):
        # Get ground truth annotations from the batch
        gt_pose = batch['pose'].to(device)
        gt_betas = batch['betas'].to(device)
        gt_vertices = smpl_neutral(betas=gt_betas, body_pose=gt_pose[:, 3:], global_orient=gt_pose[:, :3]).vertices
        images = batch['img'].to(device)
        gender = batch['gender'].to(device)
        curr_batch_size = images.shape[0]
        
        with torch.no_grad():
            pred_rotmat, pred_betas, pred_camera = model(images)
            pred_output = smpl_neutral(betas=pred_betas, body_pose=pred_rotmat[:,1:], global_orient=pred_rotmat[:,0].unsqueeze(1), pose2rot=False)
            pred_vertices = pred_output.vertices

        if save_results:
            rot_pad = torch.tensor([0,0,1], dtype=torch.float32, device=device).view(1,3,1)
            rotmat = torch.cat((pred_rotmat.view(-1, 3, 3), rot_pad.expand(curr_batch_size * 24, -1, -1)), dim=-1)
            pred_pose = tgm.rotation_matrix_to_angle_axis(rotmat).contiguous().view(-1, 72)
            smpl_pose[step * batch_size:step * batch_size + curr_batch_size, :] = pred_pose.cpu().numpy()
            smpl_betas[step * batch_size:step * batch_size + curr_batch_size, :]  = pred_betas.cpu().numpy()
            smpl_camera[step * batch_size:step * batch_size + curr_batch_size, :]  = pred_camera.cpu().numpy()
        
        # Orientation evaluation
        if eval_orientation:
            # Get ground truth orientation (already stored in gt_pose)
            gt_rotvec = torch.zeros(curr_batch_size,24,3) # Have to have an array of this shape to input into the rotation object (from 32,72 to 32,24,3)
            i = 0
            for row in gt_pose:
                gt_rotvec[i] = torch.reshape(row,(24, -1))
                i+=1

            gt_euler = np.zeros((curr_batch_size, 24, 3)) # Using numpy here because it works with the rotation library
            i = 0
            for row in gt_rotvec:
                r = R.from_rotvec(row)
                gt_euler[i] = R.as_euler(r, 'xyz', degrees=True)
                i+=1
            
            
            # Get Euler representation of the predictions too:
            
            pred_euler = np.zeros((curr_batch_size,24,3)) # Has to be a numpy array because it works with Rotation

            # For each row in pred_rotmat convert it to a Rotation object and write it into a corresponding
            # row in pred_euler as Euler angles

            i=0
            for row in pred_rotmat:
                r = R.from_dcm(row.cpu())
                pred_euler[i] = R.as_euler(r, 'xyz', degrees=True)
                i+=1
                
            orientation_error = torch.sqrt((torch.from_numpy(gt_euler).to(device) -
                                            torch.from_numpy(pred_euler).to(device))**2).sum(dim=-1).mean(dim=-1)
            print("orientation_error")
            print(orientation_error)
            print()
            
            moe[step * batch_size:step * batch_size + curr_batch_size] = orientation_error.cpu()
            
        # 3D pose evaluation
        if eval_pose:
            # Regressor broadcasting
            J_regressor_batch = J_regressor[None, :].expand(pred_vertices.shape[0], -1, -1).to(device)
            # Get 14 ground truth joints
            if 'h36m' in dataset_name or 'mpi-inf' in dataset_name:
                gt_keypoints_3d = batch['pose_3d'].cuda()
                gt_keypoints_3d = gt_keypoints_3d[:, joint_mapper_gt, :-1]
            # For 3DPW get the 14 common joints from the rendered shape
            else:
                gt_vertices = smpl_male(global_orient=gt_pose[:,:3], body_pose=gt_pose[:,3:], betas=gt_betas).vertices 
                gt_vertices_female = smpl_female(global_orient=gt_pose[:,:3], body_pose=gt_pose[:,3:], betas=gt_betas).vertices 
                gt_vertices[gender==1, :, :] = gt_vertices_female[gender==1, :, :]
                gt_keypoints_3d = torch.matmul(J_regressor_batch, gt_vertices)
                gt_pelvis = gt_keypoints_3d[:, [0],:].clone()
                gt_keypoints_3d = gt_keypoints_3d[:, joint_mapper_h36m, :]
                gt_keypoints_3d = gt_keypoints_3d - gt_pelvis 


            # Get 14 predicted joints from the mesh
            pred_keypoints_3d = torch.matmul(J_regressor_batch, pred_vertices)
            if save_results:
                pred_joints[step * batch_size:step * batch_size + curr_batch_size, :, :]  = pred_keypoints_3d.cpu().numpy()
            pred_pelvis = pred_keypoints_3d[:, [0],:].clone()
            pred_keypoints_3d = pred_keypoints_3d[:, joint_mapper_h36m, :]
            pred_keypoints_3d = pred_keypoints_3d - pred_pelvis 

            # Absolute error (MPJPE)
            error = torch.sqrt(((pred_keypoints_3d - gt_keypoints_3d) ** 2).sum(dim=-1)).mean(dim=-1).cpu().numpy()
            mpjpe[step * batch_size:step * batch_size + curr_batch_size] = error

            # Reconstuction_error
            r_error = reconstruction_error(pred_keypoints_3d.cpu().numpy(), gt_keypoints_3d.cpu().numpy(), reduction=None)
            recon_err[step * batch_size:step * batch_size + curr_batch_size] = r_error


        # If mask or part evaluation, render the mask and part images
        if eval_masks or eval_parts:
            mask, parts = renderer(pred_vertices, pred_camera)

        # Mask evaluation (for LSP)
        if eval_masks:
            center = batch['center'].cpu().numpy()
            scale = batch['scale'].cpu().numpy()
            # Dimensions of original image
            orig_shape = batch['orig_shape'].cpu().numpy()
            for i in range(curr_batch_size):
                # After rendering, convert imate back to original resolution
                pred_mask = uncrop(mask[i].cpu().numpy(), center[i], scale[i], orig_shape[i]) > 0
                # Load gt mask
                gt_mask = cv2.imread(os.path.join(annot_path, batch['maskname'][i]), 0) > 0
                # Evaluation consistent with the original UP-3D code
                accuracy += (gt_mask == pred_mask).sum()
                pixel_count += np.prod(np.array(gt_mask.shape))
                for c in range(2):
                    cgt = gt_mask == c
                    cpred = pred_mask == c
                    tp[c] += (cgt & cpred).sum()
                    fp[c] +=  (~cgt & cpred).sum()
                    fn[c] +=  (cgt & ~cpred).sum()
                f1 = 2 * tp / (2 * tp + fp + fn)

        # Part evaluation (for LSP)
        if eval_parts:
            center = batch['center'].cpu().numpy()
            scale = batch['scale'].cpu().numpy()
            orig_shape = batch['orig_shape'].cpu().numpy()
            for i in range(curr_batch_size):
                pred_parts = uncrop(parts[i].cpu().numpy().astype(np.uint8), center[i], scale[i], orig_shape[i])
                # Load gt part segmentation
                gt_parts = cv2.imread(os.path.join(annot_path, batch['partname'][i]), 0)
                # Evaluation consistent with the original UP-3D code
                # 6 parts + background
                for c in range(7):
                   cgt = gt_parts == c
                   cpred = pred_parts == c
                   cpred[gt_parts == 255] = 0
                   parts_tp[c] += (cgt & cpred).sum()
                   parts_fp[c] +=  (~cgt & cpred).sum()
                   parts_fn[c] +=  (cgt & ~cpred).sum()
                gt_parts[gt_parts == 255] = 0
                pred_parts[pred_parts == 255] = 0
                parts_f1 = 2 * parts_tp / (2 * parts_tp + parts_fp + parts_fn)
                parts_accuracy += (gt_parts == pred_parts).sum()
                parts_pixel_count += np.prod(np.array(gt_parts.shape))

        # Print intermediate results during evaluation
        if step % log_freq == log_freq - 1:
            if eval_pose:
                print('MPJPE: ' + str(1000 * mpjpe[:step * batch_size].mean()))
                print('Reconstruction Error: ' + str(1000 * recon_err[:step * batch_size].mean()))
                print()
            if eval_masks:
                print('Accuracy: ', accuracy / pixel_count)
                print('F1: ', f1.mean())
                print()
            if eval_parts:
                print('Parts Accuracy: ', parts_accuracy / parts_pixel_count)
                print('Parts F1 (BG): ', parts_f1[[0,1,2,3,4,5,6]].mean())
                print()
            if eval_orientation:
                print('Orientation error: ' + str(moe[:step * batch_size].mean()))

    # Save reconstructions to a file for further processing
    if save_results:
        np.savez(result_file, pred_joints=pred_joints, pose=smpl_pose, betas=smpl_betas, camera=smpl_camera)
    # Print final results during evaluation
    print('*** Final Results ***')
    print()
    if eval_pose:
        print('MPJPE: ' + str(1000 * mpjpe.mean()))
        print('Reconstruction Error: ' + str(1000 * recon_err.mean()))
        print()
    if eval_masks:
        print('Accuracy: ', accuracy / pixel_count)
        print('F1: ', f1.mean())
        print()
    if eval_parts:
        print('Parts Accuracy: ', parts_accuracy / parts_pixel_count)
        print('Parts F1 (BG): ', parts_f1[[0,1,2,3,4,5,6]].mean())
        print()
    if eval_orientation:
        print('Orientation Error: ' + str(moe.mean()))

In [2]:
if __name__ == '__main__':
    
    # python3 eval.py --checkpoint=data/model_checkpoint.pt --dataset=h36m-p1 --log_freq=20 // example code
    args = parser.parse_args(['--checkpoint=data/model_checkpoint.pt','--dataset=3dpw', '--log_freq=20'])
    # Here we inserted our own arguments list
    
    model = hmr(config.SMPL_MEAN_PARAMS)
    checkpoint = torch.load(args.checkpoint)
    model.load_state_dict(checkpoint['model'], strict=False)
    model.eval()

    # Setup evaluation dataset
    dataset = BaseDataset(None, args.dataset, is_train=False)
    # Run evaluation
    run_evaluation(model, args.dataset, dataset, args.result_file,
                   batch_size=args.batch_size,
                   shuffle=args.shuffle,
                   log_freq=args.log_freq)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth
Eval:   0%|          | 1/1110 [00:36<11:17:14, 36.64s/it]

orientation_error
tensor([36.5759, 36.2161, 35.9665, 35.9561, 35.8158, 35.8998, 35.8793, 35.8858,
        35.9649, 35.7993, 35.8724, 35.8524, 35.7611, 35.8862, 36.0011, 36.0329,
        36.0654, 36.0372, 35.7090, 35.6074, 35.6285, 35.4310, 35.1585, 35.2144,
        35.0660, 34.6617, 34.2508, 34.1946, 34.5147, 34.6075, 34.4772, 33.7555],
       device='cuda:0', dtype=torch.float64)

orientation_error
tensor([33.6882, 34.0760, 34.9237, 20.7577, 21.2496, 22.0872, 22.2228, 22.3118,
        21.6972, 20.9507, 20.3335, 19.8684, 20.3610, 19.9275, 20.0725, 19.6807,
        18.7825, 18.2433, 18.7598, 19.9271, 20.6324, 21.8104, 21.5735, 20.9459,
        19.1617, 19.7984, 19.6054, 19.0759, 19.1485, 18.8991, 18.5065, 17.3978],
       device='cuda:0', dtype=torch.float64)



Eval:   0%|          | 3/1110 [00:38<5:40:06, 18.43s/it] 

orientation_error
tensor([17.2138, 17.6814, 17.9307, 18.0189, 18.4331, 19.0092, 19.1053, 20.8547,
        20.1954, 19.6984, 17.9647, 17.0665, 17.0962, 17.0021, 16.9976, 31.6662,
        31.5200, 31.3736, 31.5830, 31.4137, 31.8998, 32.8919, 34.6931, 36.5171,
        37.8031, 37.5326, 37.5714, 37.7916, 37.4355, 37.3820, 36.4436, 35.8870],
       device='cuda:0', dtype=torch.float64)



Eval:   0%|          | 4/1110 [00:39<4:02:15, 13.14s/it]

...

orientation_error
tensor([23.7338, 25.2178, 24.6141, 22.2787, 22.1691, 21.1275, 21.8563, 22.9745,
        22.1941, 22.1920, 22.8354, 23.4603, 24.0880, 25.5365, 25.4100, 24.8566,
        24.0144, 25.4563, 25.4972, 25.5208, 26.3695, 25.1184, 25.4015, 25.4933,
        24.7076, 24.6623, 22.9913, 22.2866, 21.5588, 21.7413, 22.3415, 22.9814],
       device='cuda:0', dtype=torch.float64)

orientation_error
tensor([23.3481, 22.8331, 22.6573, 22.8590, 22.7474, 22.3722, 22.4344, 22.1227,
        22.6968, 22.7503, 23.6454, 24.6465, 25.1758, 24.1325, 23.7672, 23.9815,
        24.2860, 22.8141, 22.2568, 23.4483, 22.5189, 22.3363, 23.9334, 25.9163,
        24.2152, 24.9284, 25.5895, 26.2608, 25.7157, 26.7158, 25.9038, 27.3515],
       device='cuda:0', dtype=torch.float64)



Eval: 100%|█████████▉| 1109/1110 [11:52<00:00,  7.13it/s]

orientation_error
tensor([28.3544, 30.4808, 30.3280, 33.3973, 34.5051, 36.8159, 38.7361, 41.2268,
        49.9205, 49.7851, 49.8190, 49.6826, 50.1614, 50.0169, 50.0999, 51.1859,
        49.7064, 48.5724, 48.6221, 47.4222, 36.8103, 37.3804, 37.3857, 35.0589,
        32.9712, 30.8358, 25.1252, 26.4135, 26.4651, 26.4419, 24.8681, 25.7697],
       device='cuda:0', dtype=torch.float64)

orientation_error
tensor([25.6590, 25.3603, 25.1070, 25.4939, 25.6336, 25.3966, 24.9788, 25.3156,
        26.1491, 25.8186, 26.2396, 26.3791, 26.1285, 41.4524, 40.9973, 39.7262,
        38.9048, 22.9330, 37.1163, 37.2088, 36.2543, 36.2167, 36.1838, 36.5195,
        37.4312, 38.4229, 39.5530, 40.0373, 40.4576, 26.3344, 26.2738, 25.8840],
       device='cuda:0', dtype=torch.float64)



Eval: 100%|██████████| 1110/1110 [11:52<00:00,  7.58it/s]

orientation_error
tensor([25.8907, 25.9221, 25.8416, 25.8340, 26.0514, 26.1199, 26.3219, 26.4102,
        26.3512, 26.4559, 26.8296, 27.0161, 27.0098, 27.0246, 27.1904, 27.2047,
        27.2286, 27.1539, 27.1334, 27.2318, 27.1115, 27.1495, 26.9679, 27.0063,
        26.9473, 26.9912, 26.7678], device='cuda:0', dtype=torch.float64)

*** Final Results ***

MPJPE: 96.94085740828478
Reconstruction Error: 59.285746753744455

Orientation Error: 31.627731715879733


In [ ]:
import torch
from torch.utils.data import DataLoader
import numpy as np
import cv2
import os
import argparse
import json
from collections import namedtuple
from tqdm import tqdm
import torchgeometry as tgm

import config
import constants
from models import hmr, SMPL
from datasets import BaseDataset
from utils.imutils import uncrop
from utils.pose_utils import reconstruction_error
from utils.part_utils import PartRenderer

from scipy.spatial.transform import Rotation as R

pred_rotmat = torch.rand(32,24,3,3, device='cuda')
pred_euler = np.zeros((32,24,3)) # Has to be a numpy array because it works with Rotation

# For each row in pred_rotmat convert it to a Rotation object and write it into a corresponding
# row in pred_euler as Euler angles

i=0
for row in pred_rotmat:
    r = R.from_dcm(row.cpu())
    pred_euler[i] = R.as_euler(r, 'xyz', degrees=True)
    i+=1
    
pred_euler

In [ ]:
gt_pose = torch.rand(32,72)

gt_rotvec = torch.zeros(32,24,3) # Have to have an array of this shape to input into the rotation object (from 32,72 to 32,24,3)
i = 0
for row in gt_pose:
    gt_rotvec[i] = torch.reshape(row,(24, -1))
    i+=1

gt_euler = np.zeros((32, 24, 3)) # Using numpy here because it works with the rotation library
i = 0
for row in gt_rotvec:
    r = R.from_rotvec(row)
    gt_euler[i] = R.as_euler(r, 'xyz', degrees=True)
    i+=1
    
np.shape(gt_euler - pred_euler)

In [ ]:
(gt_euler - pred_euler).mean()

In [ ]:
row = 0
pred_rotmat = torch.rand(32,24,3,3)
pred_rotmat[row].size()

In [ ]:
import torch
import numpy as np

torch.set_printoptions(profile="full")
np.set_printoptions(threshold=np.inf)

gt_pose = torch.rand(32,72)
a

#np.apply_along_axis(np.reshape( ,(24, -1)), -1, a)

gt_orientation_reshaped = torch.zeros(32,24,3) # Have to reshape from 32,72 to 32,24,3 to input into the rotation object
i = 0

for row in gt_pose:
    gt_orientation_reshaped[i] = torch.reshape(row,(24, -1))
    i+=1
    
gt_orientation_as_matrix = np.zeros((32, 24, 3, 3)) # Using numpy here because it works with the rotation library
i = 0

for row in gt_orientation_reshaped:
    r = R.from_rotvec(row)
    gt_orientation_as_matrix[i] = r.as_dcm()
    i+=1

In [ ]:
#moe = np.zeros(32)
pred_rotmat = torch.rand(32,24,3,3)

orientation_error = pred_rotmat - torch.as_tensor(gt_orientation_as_matrix, dtype=torch.float, device='cpu')
#moe = orientation_error

In [ ]:
orientation_error.mean(-1).mean(-1).mean(-1).cpu().numpy()

In [ ]:
b[0].size()

In [ ]:
np.shape(r_matrix)

In [ ]:
r_matrix

In [ ]:
a.size()

In [ ]:
b.size()

In [ ]:
a

In [ ]:
b